In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
# Set the path to the data
data_dir = '/content/drive/My Drive/SRI'
csv_file = '/content/drive/My Drive/SRI/english.csv'

In [4]:
# Load the CSV file
data = pd.read_csv(csv_file)

In [5]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data['image'], data['label'], test_size=0.2, random_state=42)


In [6]:
# Preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.1, horizontal_flip=True)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=data.loc[X_train.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=64,
    class_mode='categorical'
)

Found 2728 validated image filenames belonging to 62 classes.


In [7]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=data.loc[X_val.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=64,
    class_mode='categorical'
)

Found 682 validated image filenames belonging to 62 classes.


In [8]:
# Load the pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(400, 300, 3))


29084464/29084464 [==============================] - 0s 0us/step


In [9]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(62, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

In [11]:
# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
# Train the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
model.fit(train_generator, epochs=100, validation_data=val_generator, callbacks=[early_stopping])


Epoch 1/100
43/43 [==============================] - 1525s 35s/step - loss: 4.2162 - accuracy: 0.0260 - val_loss: 3.8704 - val_accuracy: 0.1716
Epoch 2/100
43/43 [==============================] - 134s 3s/step - loss: 3.7485 - accuracy: 0.0872 - val_loss: 3.1254 - val_accuracy: 0.3343
Epoch 3/100
43/43 [==============================] - 132s 3s/step - loss: 3.2493 - accuracy: 0.1536 - val_loss: 2.4148 - val_accuracy: 0.5191
Epoch 4/100
43/43 [==============================] - 138s 3s/step - loss: 2.7366 - accuracy: 0.2504 - val_loss: 2.0082 - val_accuracy: 0.5469
Epoch 5/100
43/43 [==============================] - 132s 3s/step - loss: 2.4523 - accuracy: 0.3229 - val_loss: 1.7306 - val_accuracy: 0.5938
Epoch 6/100
43/43 [==============================] - 133s 3s/step - loss: 2.2650 - accuracy: 0.3493 - val_loss: 1.5058 - val_accuracy: 0.6437
Epoch 7/100
43/43 [==============================] - 138s 3s/step - loss: 2.0642 - accuracy: 0.3959 - val_loss: 1.3931 - val_accuracy: 0.6701
Epoc

In [13]:
# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=data.loc[X_val.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=64,
    class_mode='categorical'
)

Found 682 validated image filenames belonging to 62 classes.


In [14]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.4f}')

11/11 [==============================] - 14s 1s/step - loss: 0.5629 - accuracy: 0.8138
Test Accuracy: 0.8138


In [15]:
# Save the trained model
import joblib
joblib.dump(model, '/content/drive/My Drive/SRI/DenseNet121_model3.pkl')

['/content/drive/My Drive/SRI/DenseNet121_model3.pkl']